## Imports

In [15]:
import requests
import time
import json
import pandas as pd
import folium
%store -r google_places_API_Key

## Get all H-E-B stores

In [11]:
API_KEY = google_places_API_Key
BASE_URL = 'https://maps.googleapis.com/maps/api/place/textsearch/json?'
query = "H-E-B grocery in Texas"

all_stores = []

params = {
    'query': query,
    'key': API_KEY
}

while True:
    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        all_stores.extend(data['results'])
        
        # Check if there are more pages of results
        next_page_token = data.get('next_page_token')
        if next_page_token:
            # The token becomes valid after a short delay, so we wait for a few seconds
            time.sleep(2)
            params['pagetoken'] = next_page_token
        else:
            # If no more results, break out of the loop
            break
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
        break

# Save the results to a JSON file
with open('heb_stores.json', 'w') as outfile:
    json.dump(all_stores, outfile)

In [13]:
# Load JSON data
with open('heb_stores.json', 'r') as file:
    data = json.load(file)

# Convert to DataFrame
df = pd.json_normalize(data)

# Keep only the relevant columns
df = df[['name', 'geometry.location.lat', 'geometry.location.lng', 'formatted_address']]

# Rename the columns for clarity
df.columns = ['Name', 'Latitude', 'Longitude', 'Address']

# Display the DataFrame
print(df.head())

          Name   Latitude  Longitude   
0        H-E-B  30.612640 -96.318849  \
1        H-E-B  32.411814 -96.839792   
2  H-E-B plus!  32.520607 -97.351868   
3        H-E-B  32.433208 -99.757791   
4        H-E-B  32.457650 -97.729731   

                                                      Address  
0  1900 Texas Ave S, College Station, TX 77840, United States  
1           800 N Hwy 77, Waxahachie, TX 75165, United States  
2     165 NW John Jones Dr, Burleson, TX 76028, United States  
3            1345 Barrow St, Abilene, TX 79605, United States  
4     3804 East US Hwy 377, Granbury, TX 76049, United States  


In [16]:
# Create a base map
m = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=6)

# Add markers for each store
for idx, row in df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"<strong>Name:</strong> {row['Name']}<br><strong>Address:</strong> {row['Address']}<br><strong>Latitude:</strong> {row['Latitude']}<br><strong>Longitude:</strong> {row['Longitude']}",
        icon=folium.Icon(icon="cloud"),
    ).add_to(m)

# Display the map
m